# The Determinants of Small Shareholder Victory

In [15]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')

In [16]:
%%stata

clear all
set more off
set varabbrev off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"

In [17]:
%%stata

import delimited using "$PROCESSED_DATA/proposals_panel.csv", clear

* Parse date
gen day = date(date, "YMD")
format day %td
gen month = month(day)
gen quarter = quarter(day)

gen year = year(day)
encode gecko_id, gen(token)
encode space,    gen(dao)

* Dependent variables
local wins non_whale_victory_vn non_whale_victory_vp non_whale_victory_vp_vn

* Independent variables
local proposal_char multi_choices weighted quorum have_discussion delegation
local delegate_char whale_from_delegation_rate non_whale_from_delegation_rate ///
                    whale_to_delegation_rate non_whale_to_delegation_rate
local vote_char whale_vs_hhi non_whale_vs_hhi whale_vn_hhi non_whale_vn_hhi

* Label variables
label var non_whale_victory_vn "\${\it Small Win}^{vn}_{i,t}\$"
label var non_whale_victory_vp "\${\it Small Win}^{vp}_{i,t}\$"
label var non_whale_victory_vp_vn "\${\it Small Win}^{vp,vn}_{i,t}\$"
label var weighted "\${\it Weighted}_{i,t}\$"
label var quorum "\${\it Quorum}_{i,t}\$"
label var multi_choices "\${\it Multiple Choices}_{i,t}\$"
label var have_discussion "\${\it Discussion}_{i,t}\$"
label var delegation "\${\it Delegation}_{i,t}\$"
label var non_whale_from_delegation_rate "\$\textit{Delegator}_{i,t}^{\textit{Small}}\$"
label var whale_from_delegation_rate "\$\textit{Delegator}_{i,t}^{\textit{Whale}}\$"
label var non_whale_to_delegation_rate "\$\textit{Delegatee}_{i,t}^{\textit{Small}}\$"
label var whale_to_delegation_rate "\$\textit{Delegatee}_{i,t}^{\textit{Whale}}\$"
label var non_whale_vs_hhi "\$\textit{HHI}_{i,t}^{\textit{Small,vs}}\$"
label var whale_vs_hhi "\$\textit{HHI}_{i,t}^{\textit{Whale,vs}}\$"
label var non_whale_vn_hhi "\$\textit{HHI}_{i,t}^{\textit{Small,vn}}\$"
label var whale_vn_hhi "\$\textit{HHI}_{i,t}^{\textit{Whale,vn}}\$"

## Summary Statistics

In [18]:
%%stata

******************************************************
* SUMMARY STATISTICS (before regression)
******************************************************
* Summarize all key variables
summarize `wins' `proposal_char' `delegate_char' `vote_char', detail

* Export a nice LaTeX summary table with mean, SD, p10, median, p90
estpost summarize `wins' `proposal_char' `delegate_char' `vote_char', detail
esttab using "$TABLES/summary_stats.tex", replace ///
    cells("count(fmt(0)) mean(fmt(3)) sd(fmt(3)) p10(fmt(3)) p50(fmt(3)) p90(fmt(3))") ///
    collabels("Number of Obs." "Mean" "Std. Dev." "P10" "Median" "P90") ///
    nonumber nomtitles noobs label ///
    alignment(D{.}{.}{-1}) booktabs ///
    compress nofloat nostar ///
    substitute("\_" "_")                     ///
    title("Summary Statistics of Variables") ///
    prehead("\begin{tabular}{l*{6}{c}}\toprule") ///
    posthead("\hline") ///
    postfoot("\bottomrule\end{tabular}")

## Proposal Characteristics

In [19]:
%%stata

******************************************************
* LOGISTIC REGRESSIONS
******************************************************

eststo clear

foreach y of local wins {
    * Logit regression with SEs clustered by DAO
    logit `y' `proposal_char', vce(cluster dao)
    eststo `y'_p
}

* Export LaTeX table
esttab                                                     ///
    non_whale_victory_vn_p                                 ///
    non_whale_victory_vp_p                                 ///
    non_whale_victory_vp_vn_p                              ///
    using "$TABLES/reg_logit_win_proposal.tex", replace    ///
    se star(* 0.10 ** 0.05 *** 0.01)                       ///
    b(%9.3f) se(%9.2f)                                     ///
    label nogaps booktabs nonotes noomitted eqlabels(none) ///
    mtitles("\${\it Small Win}^{vn}_{i,t}\$"               ///
    "\${\it Small Win}^{vp}_{i,t}\$"                       ///  
    "\${\it Small Win}^{vp,vn}_{i,t}\$")                   ///
    substitute("\_" "_")                                   ///
    stats(N r2_p,                                          ///
        fmt(%9.0fc %9.3f)                                  ///
        labels("Observations" "Pseudo R²"))

## Delegation Characteristics

In [22]:
%%stata

******************************************************
* LOGISTIC REGRESSIONS
******************************************************

eststo clear

foreach y of local wins {
    * Logit regression with SEs clustered by DAO
    logit `y' whale_from_delegation_rate non_whale_from_delegation_rate, vce(cluster dao)
    eststo `y'_f

    logit `y' whale_to_delegation_rate non_whale_to_delegation_rate, vce(cluster dao)
    eststo `y'_t
}

* Export LaTeX table
esttab                                                      ///
    non_whale_victory_vn_f                                  ///
    non_whale_victory_vn_t                                  ///
    non_whale_victory_vp_f                                  ///
    non_whale_victory_vp_t                                  ///
    non_whale_victory_vp_vn_f                               ///
    non_whale_victory_vp_vn_t                               ///
    using "$TABLES/reg_logit_win_delegate.tex", replace     ///
    se star(* 0.10 ** 0.05 *** 0.01)                        ///
    b(%9.3f) se(%9.2f)                                      ///
    label nogaps booktabs nonotes noomitted eqlabels(none)  ///
    mgroups("\${\it Small Win}^{vn}_{i,t}\$"                ///
    "\${\it Small Win}^{vp}_{i,t}\$"                        ///  
    "\${\it Small Win}^{vp,vn}_{i,t}\$",                    ///
    span                                                    ///
    pattern(1 0 1 0 1 0)                                    ///
    prefix(\multicolumn{@span}{c}{)                         ///
    suffix(})                                               ///
    erepeat(\cmidrule(lr){@span}) )                         ///
    substitute("\_" "_")                                    ///
    stats(N r2_p,                                           ///
        fmt(%9.0fc %9.3f)                                   ///
        labels("Observations" "Pseudo R²"))

## Voting Characteristics

In [23]:
%%stata

******************************************************
* LOGISTIC REGRESSIONS
******************************************************

eststo clear

logit non_whale_victory_vn whale_vn_hhi non_whale_vn_hhi 
eststo non_whale_victory_vn_h

logit non_whale_victory_vp whale_vs_hhi non_whale_vs_hhi
eststo non_whale_victory_vp_h

logit non_whale_victory_vp_vn whale_vs_hhi non_whale_vn_hhi 
eststo non_whale_victory_vp_vn_h


* Export LaTeX table
esttab                                                     ///
    non_whale_victory_vn_h                                 ///
    non_whale_victory_vp_h                                 ///
    non_whale_victory_vp_vn_h                              ///
    using "$TABLES/reg_logit_win_vote.tex", replace    ///
    se star(* 0.10 ** 0.05 *** 0.01)                       ///
    b(%9.3f) se(%9.2f)                                     ///
    label nogaps booktabs nonotes noomitted eqlabels(none) ///
    mtitles("\${\it Small Win}^{vn}_{i,t}\$"               ///
    "\${\it Small Win}^{vp}_{i,t}\$"                       ///  
    "\${\it Small Win}^{vp,vn}_{i,t}\$")                   ///
    substitute("\_" "_")                                   ///
    stats(N r2_p,                                          ///
        fmt(%9.0fc %9.3f)                                  ///
        labels("Observations" "Pseudo R²"))